# MongoDB Atlas Indexes


<br>




In [1]:
import pymongo
import os
from pymongo import MongoClient

password = os.environ['PASSWORD']

client = MongoClient('mongodb+srv://pepe:' + password +'@cluster0-dajh0.mongodb.net/test?retryWrites=true&w=majority')
server_info = client.server_info()

In [2]:
import pandas

database_names = client.database_names()
df = pandas.DataFrame(database_names, columns=['Database Names'])
print(df)

       Database Names
0               admin
1              config
2               local
3       sample_airbnb
4    sample_analytics
5   sample_geospatial
6        sample_mflix
7     sample_supplies
8     sample_training
9  sample_weatherdata


In [3]:
db = client.sample_airbnb
collection_names = db.collection_names()
df = pandas.DataFrame(collection_names, columns=['Collection Names'])
print(df)

     Collection Names
0  listingsAndReviews


Getting index information

In [5]:
import pprint

collection = db.listingsAndReviews
for i in dir(collection):
    if 'index' in i:
        print(i)

_Collection__create_index
create_index
create_indexes
drop_index
drop_indexes
ensure_index
index_information
list_indexes
reindex


## Index Information



In [7]:

index_information = collection.index_information()
pprint.pprint(index_information)


{'_id_': {'key': [('_id', 1)],
          'ns': 'sample_airbnb.listingsAndReviews',
          'v': 2},
 'address.location_2dsphere': {'2dsphereIndexVersion': 3,
                               'background': True,
                               'key': [('address.location', '2dsphere')],
                               'ns': 'sample_airbnb.listingsAndReviews',
                               'v': 2},
 'name_1': {'background': True,
            'key': [('name', 1)],
            'ns': 'sample_airbnb.listingsAndReviews',
            'v': 2},
 'property_type_1_room_type_1_beds_1': {'background': True,
                                        'key': [('property_type', 1),
                                                ('room_type', 1),
                                                ('beds', 1)],
                                        'ns': 'sample_airbnb.listingsAndReviews',
                                        'v': 2}}


In [8]:
# iterate the dict object returned by the method call 
for key, val in index_information.items(): 
    print(key, "--", val)

_id_ -- {'v': 2, 'key': [('_id', 1)], 'ns': 'sample_airbnb.listingsAndReviews'}
property_type_1_room_type_1_beds_1 -- {'v': 2, 'key': [('property_type', 1), ('room_type', 1), ('beds', 1)], 'background': True, 'ns': 'sample_airbnb.listingsAndReviews'}
name_1 -- {'v': 2, 'key': [('name', 1)], 'ns': 'sample_airbnb.listingsAndReviews', 'background': True}
address.location_2dsphere -- {'v': 2, 'key': [('address.location', '2dsphere')], 'ns': 'sample_airbnb.listingsAndReviews', 'background': True, '2dsphereIndexVersion': 3}


## List indexes




In [10]:
index_cursor = collection.list_indexes() 
# iterate the pymongo.command_cursor.CommandCursor object 
for index in index_cursor: 
# print the bson.son.SON object for each iteration 
    print ("\n") 
    print ("index.keys():", index.keys()) 
    print ("NAME:", index["name"]) # index name 
    print ("VERSION:", index["v"]) # index version 

# iterate the index bson object's key-value pairs 
# using the object's items() method 
for key, val in index.items(): 
    print (key, "--->", val, "-- type:", type(val))



index.keys(): ['v', 'key', 'name', 'ns']
NAME: _id_
VERSION: 2


index.keys(): ['v', 'key', 'name', 'background', 'ns']
NAME: property_type_1_room_type_1_beds_1
VERSION: 2


index.keys(): ['v', 'key', 'name', 'ns', 'background']
NAME: name_1
VERSION: 2


index.keys(): ['v', 'key', 'name', 'ns', 'background', '2dsphereIndexVersion']
NAME: address.location_2dsphere
VERSION: 2
v ---> 2 -- type: <class 'int'>
key ---> SON([('address.location', '2dsphere')]) -- type: <class 'bson.son.SON'>
name ---> address.location_2dsphere -- type: <class 'str'>
ns ---> sample_airbnb.listingsAndReviews -- type: <class 'str'>
background ---> True -- type: <class 'bool'>
2dsphereIndexVersion ---> 3 -- type: <class 'int'>


## Execution Stats




In [12]:
for i in collection.find({'bedrooms': 3, 'address': {'country' : 'Portugal'}}).limit(2):
    pprint.pprint(i)

java.lang.RuntimeException: NameError: name 'collection' is not defined

In [13]:
execution_stats = collection.find().explain()['executionStats']
pprint.pprint(execution_stats)

{'allPlansExecution': [],
 'executionStages': {'advanced': 5555,
                     'direction': 'forward',
                     'docsExamined': 5555,
                     'executionTimeMillisEstimate': 0,
                     'isEOF': 1,
                     'nReturned': 5555,
                     'needTime': 1,
                     'needYield': 0,
                     'restoreState': 43,
                     'saveState': 43,
                     'stage': 'COLLSCAN',
                     'works': 5557},
 'executionSuccess': True,
 'executionTimeMillis': 2,
 'nReturned': 5555,
 'totalDocsExamined': 5555,
 'totalKeysExamined': 0}
